In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam          
from tensorflow.keras.applications import MobileNetV2 
from tensorflow.keras.layers import GlobalAveragePooling2D

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sn
import pandas as pd
from sklearn.metrics import confusion_matrix
import os

In [ ]:
# CONFIGURATION DES PARAMÈTRES
DATA_DIR = r"data/"

if DATA_DIR == "data":
    raise ValueError("Veuillez modifier la variable 'DATA_DIR' pour pointer vers votre dossier de données.")
if not os.path.exists(DATA_DIR):
    raise FileNotFoundError(f"Le dossier spécifié n'existe pas : {DATA_DIR}")

# Paramètres du modèle
IMG_HEIGHT = 224 
IMG_WIDTH = 224   
IMG_SIZE = (IMG_HEIGHT, IMG_WIDTH)
BATCH_SIZE = 32
EPOCHS = 50     
VALIDATION_SPLIT = 0.2
LEARNING_RATE = 0.0001 

print(" 2. CHARGEMENT ET PRÉTRAITEMENT DES DONNÉES ")

# Chargement des données d'entraînement
train_dataset = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=VALIDATION_SPLIT,
    subset="training",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

# Chargement des données de validation
val_dataset = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=VALIDATION_SPLIT,
    subset="validation",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

# Récupérer les noms des classes
class_names = train_dataset.class_names
num_classes = len(class_names)
print(f"Classes trouvées : {class_names} (Total: {num_classes})")


# DATA AUGMENTATION (ET NORMALISATION) 
print("\n 3. DATA AUGMENTATION (RENFORCÉE) ")

data_augmentation = Sequential(
    [
        layers.RandomFlip("horizontal", input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
        layers.RandomBrightness(0.2),  
        layers.RandomContrast(0.2)     
    ],
    name="data_augmentation"
)

# Optimiser les performances
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.cache().prefetch(buffer_size=AUTOTUNE)

print("\n 4. CONSTRUCTION DU MODÈLE (TRANSFER LEARNING) ")

# Charger le modèle de base (MobileNetV2) pré-entraîné sur ImageNet
base_model = MobileNetV2(
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
    include_top=False,
    weights='imagenet'
)

# Geler le modèle de base
base_model.trainable = False

# Créer notre nouveau modèle
model = Sequential([
    # Couche d'augmentation
    data_augmentation,

    # Couche de normalisation (IMPORTANT)
    # MobileNetV2 attend des pixels entre [-1, 1], pas [0, 255] ou [0, 1]
    layers.Rescaling(1./127.5, offset=-1),

    # Le modèle de base (gelé)
    base_model,

    # Nos propres couches de classification
    GlobalAveragePooling2D(), # Transforme la sortie en vecteur 1D (mieux que Flatten)
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
], name="Fruit_Classifier_TransferLearning")


# COMPILATION DU MODÈLE 
print("\n 5. COMPILATION DU MODÈLE ")

# Utiliser un optimiseur Adam avec un taux d'apprentissage plus faible
optimizer = Adam(learning_rate=LEARNING_RATE)

model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

# Afficher un résumé de l'architecture
model.summary()


# ENTRAÎNEMENT DU MODÈLE 
print("\n 6. ENTRAÎNEMENT DU MODÈLE ")

# Définir le callback EarlyStopping
# Il arrêtera l'entraînement si la 'val_loss' ne s'améliore pas
# pendant 'patience' époques.
early_stopping = EarlyStopping(
    monitor='val_loss',     
    patience=5,             
    verbose=1,
    restore_best_weights=True 
) 

history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS,
    callbacks=[early_stopping] 
)


# ÉVALUATION ET VISUALISATION DES RÉSULTATS 
print("\n 7. ÉVALUATION ET VISUALISATION DES RÉSULTATS ")

# Graphiques d'Accuracy et de Perte 
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

# Utiliser le nombre d'époques *réellement* exécutées
epochs_range = range(len(acc))

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label="Accuracy (Entraînement)")
plt.plot(epochs_range, val_acc, label="Accuracy (Validation)")
plt.legend(loc='lower right')
plt.title("Accuracy (Entraînement et Validation)")

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label="Perte (Entraînement)")
plt.plot(epochs_range, val_loss, label="Perte (Validation)")
plt.legend(loc='upper right')
plt.title("Perte (Entraînement et Validation)")
plt.suptitle(f"Métriques d'entraînement (Meilleure époque: {np.argmin(val_loss)})", fontsize=16)
plt.show()


# Matrice de Confusion
print("\nCalcul de la matrice de confusion...")

# Prédire sur l'ensemble de validation complet
y_pred_probs = model.predict(val_dataset)
y_pred_classes = np.argmax(y_pred_probs, axis=1)

# Obtenir les vraies étiquettes
y_true = np.concatenate([y for x, y in val_dataset], axis=0)

# Calculer la matrice
cm = confusion_matrix(y_true, y_pred_classes)

# Afficher la matrice avec Seaborn
df_cm = pd.DataFrame(cm, index=class_names, columns=class_names)
plt.figure(figsize=(10, 7))
sn.heatmap(df_cm, annot=True, fmt='g', cmap='Blues')
plt.title(f'Matrice de Confusion\nAccuracy: {np.trace(cm) / np.sum(cm):.2%}')
plt.xlabel('Prédictions')
plt.ylabel('Vraies étiquettes')
plt.show()

print("\nScript terminé.")